In [1]:
import pandas as pd

data = pd.read_excel('disl_hackaton.xlsx')

In [4]:
import pandas as pd


In [4]:
peregon_data = pd.read_excel('PEREGON_HACKATON.xlsx')

In [5]:
peregon_data

,START_CODE,END_CODE,LEN
0,2,805,32
1,6,2,27
2,8,6,27
3,12,8,29
4,15,12,19
...,...,...,...
8969,24329,7173,9
8970,25137,5956,10
8971,25137,5985,1
8972,25612,1688,0


In [36]:
station_data = pd.read_excel('STATION_COORDS_HACKATON.xlsx')

In [42]:
station_data

,ST_ID,LATITUDE,LONGITUDE
0,2,48.4272,42.2162
1,3,48.4910,42.3610
2,4,48.3981,42.0699
3,5,48.3857,41.9546
4,6,48.5423,42.5028
...,...,...,...
15168,25876,56.5624,57.7346
15169,25877,56.6841,57.9468
15170,25878,56.8725,59.2641
15171,25879,56.4072,52.9355


In [41]:
station_data.drop_duplicates(inplace=True)

In [7]:
import overpy

def find_railway_station_osm_with_id(latitude, longitude):
    api = overpy.Overpass()

    # Формирование запроса к OSM API для поиска железнодорожной станции
    query = f"""
        [out:json];
        node(around:1000,{latitude},{longitude})[railway=station];
        out body;
    """

    result = api.query(query)

    # Проверка наличия результатов
    if result.nodes:
        # Берем первую станцию из результатов
        station = result.nodes[0]
        
        # Возвращаем имя станции и её osm_id
        return station.tags.get("name", "Железнодорожная станция"), station.id

    return None, None

def add_station_name(row):
    return find_railway_station_osm_with_id(row['LATITUDE'], row['LONGITUDE'])

station_data[['name', 'osm_id']] = station_data.apply(add_station_name, axis=1)

In [1]:
station_data

NameError: name 'station_data' is not defined

In [2]:
data

,WAGNUM,OPERDATE,ST_ID_DISL,ST_ID_DEST,TRAIN_INDEX
0,5266,2023-08-30 01:02:00,7475,61,7475-335-62
1,5266,2023-08-30 05:26:00,63,61,7475-335-62
2,5266,2023-08-30 05:05:00,7475,61,7475-335-62
3,5266,2023-08-28 23:45:00,7475,61,6999-471-7475
4,5266,2023-08-28 22:29:00,7469,61,6999-471-7475
...,...,...,...,...,...
999994,5800,2023-07-15 03:47:00,2284,2331,16069-199-2331
999995,7034,2023-07-15 03:47:00,2284,2331,16069-199-2331
999996,7034,2023-07-15 02:08:00,2276,2331,16069-199-2331
999997,5800,2023-07-15 02:08:00,2276,2331,16069-199-2331


In [30]:
import pandas as pd

data['OPERDATE'] = pd.to_datetime(data['OPERDATE'])

data['TRAIN_NUM'] = data['TRAIN_INDEX'].str.split('-').str[1]

data.sort_values(by=['OPERDATE'], inplace=True)

data['WAGNUM_CURRENT'] = data.groupby('TRAIN_INDEX')['WAGNUM'].transform(lambda x: x.nunique())

data['WAGNUM_CURRENT'] = data['WAGNUM_CURRENT'].fillna(0).astype(int)

# Определение самого раннего и самого позднего времени для каждого поезда
data['EARLIEST_TIME'] = data.groupby('TRAIN_INDEX')['OPERDATE'].transform('min')
data['LATEST_TIME'] = data.groupby('TRAIN_INDEX')['OPERDATE'].transform('max')

# Фильтрация данных между самым ранним и самым поздним временем
data['OPERATION_DURATION'] = data['LATEST_TIME'] - data['EARLIEST_TIME']

# data['ROUTE_STATIONS'] = data['ST_ID_DISL'].astype(str) + '-' + data['ST_ID_DEST'].astype(str)


In [24]:
data['WAGNUM_CURRENT'].value_counts()

WAGNUM_CURRENT
1     114180
9      95374
7      95238
2      86505
8      79644
10     76500
5      73357
3      72226
4      71576
6      68616
11     57852
13     33202
12     31185
14     11604
15      9862
20      6548
16      5954
17      5867
18      1704
19       961
21       945
36       363
22       269
23       261
26       191
0         15
Name: count, dtype: int64

In [33]:
data

,WAGNUM,OPERDATE,ST_ID_DISL,ST_ID_DEST,TRAIN_INDEX,TRAIN_NUM,WAGNUM_CURRENT,OPERATION_DURATION,ROUTE_STATIONS,EARLIEST_TIME,LATEST_TIME
339038,5100,2023-06-01 00:00:00,1268,1655,1268-677-1663,677,2,1 days 03:30:00,1268-1655,2023-06-01 00:00:00,2023-06-02 03:30:00
340706,6516,2023-06-01 00:00:00,1268,1655,1268-677-1663,677,2,1 days 03:30:00,1268-1655,2023-06-01 00:00:00,2023-06-02 03:30:00
339037,5100,2023-06-01 00:00:01,1268,1655,1268-677-1663,677,2,1 days 03:30:00,1268-1655,2023-06-01 00:00:00,2023-06-02 03:30:00
340705,6516,2023-06-01 00:00:01,1268,1655,1268-677-1663,677,2,1 days 03:30:00,1268-1655,2023-06-01 00:00:00,2023-06-02 03:30:00
88525,6394,2023-06-01 00:01:00,4576,633,20715-079-20713,079,1,0 days 06:47:01,4576-633,2023-06-01 00:01:00,2023-06-01 06:48:01
...,...,...,...,...,...,...,...,...,...,...,...
596568,9100,2023-09-01 00:00:00,1771,1771,-236-,236,7,0 days 08:28:00,1771-1771,2023-08-31 15:32:00,2023-09-01 00:00:00
56767,5103,2023-09-01 00:00:00,7076,574,17506-012-20716,012,3,0 days 09:09:00,7076-574,2023-08-31 14:51:00,2023-09-01 00:00:00
101489,4596,2023-09-01 00:00:00,7076,1138,17506-012-20716,012,3,0 days 09:09:00,7076-1138,2023-08-31 14:51:00,2023-09-01 00:00:00
543961,7349,2023-09-01 00:00:00,1771,1771,-236-,236,7,0 days 08:28:00,1771-1771,2023-08-31 15:32:00,2023-09-01 00:00:00


In [34]:
data[data["TRAIN_INDEX"] == "95-470-585"]

,WAGNUM,OPERDATE,ST_ID_DISL,ST_ID_DEST,TRAIN_INDEX,TRAIN_NUM,WAGNUM_CURRENT,OPERATION_DURATION,ROUTE_STATIONS,EARLIEST_TIME,LATEST_TIME
72058,7776,2023-06-01 15:29:00,585,585,95-470-585,470,2,18 days 22:56:00,585-585,2023-06-01 15:29:00,2023-06-20 14:25:00
72057,7776,2023-06-01 15:36:00,585,585,95-470-585,470,2,18 days 22:56:00,585-585,2023-06-01 15:29:00,2023-06-20 14:25:00
72056,10000,2023-06-02 06:13:00,585,585,95-470-585,470,2,18 days 22:56:00,585-585,2023-06-01 15:29:00,2023-06-20 14:25:00
72055,10000,2023-06-02 06:35:00,585,585,95-470-585,470,2,18 days 22:56:00,585-585,2023-06-01 15:29:00,2023-06-20 14:25:00
72054,7776,2023-06-03 10:33:00,585,585,95-470-585,470,2,18 days 22:56:00,585-585,2023-06-01 15:29:00,2023-06-20 14:25:00
72053,10000,2023-06-04 03:36:00,585,585,95-470-585,470,2,18 days 22:56:00,585-585,2023-06-01 15:29:00,2023-06-20 14:25:00
72052,10000,2023-06-04 03:59:00,585,585,95-470-585,470,2,18 days 22:56:00,585-585,2023-06-01 15:29:00,2023-06-20 14:25:00
72051,10000,2023-06-04 22:52:00,585,585,95-470-585,470,2,18 days 22:56:00,585-585,2023-06-01 15:29:00,2023-06-20 14:25:00
72050,10000,2023-06-04 23:36:00,585,585,95-470-585,470,2,18 days 22:56:00,585-585,2023-06-01 15:29:00,2023-06-20 14:25:00
72049,10000,2023-06-05 12:53:00,585,585,95-470-585,470,2,18 days 22:56:00,585-585,2023-06-01 15:29:00,2023-06-20 14:25:00


In [35]:
data.to_csv('history_data.csv')

In [11]:
data['OPERDATE'].describe()

count                           999999
mean     2023-07-15 14:53:23.292152320
min                2023-06-01 00:00:00
25%                2023-06-23 04:05:00
50%                2023-07-14 19:24:00
75%                2023-08-07 02:01:00
max                2023-09-01 00:00:00
Name: OPERDATE, dtype: object

In [2]:
import overpy
import pandas as pd

# Функция для получения отношения маршрута между двумя станциями
def get_route_relation(st_id_disl, st_id_dest):
    api = overpy.Overpass()

    query = f"""
        relation["route"="train"](if: (t["railway"] == "rail" and t["ref"] == "{st_id_disl}") or (t["railway"] == "rail" and t["ref"] == "{st_id_dest}"));
        out geom;
    """
    
    result = api.query(query)

    if result.relations:
        return result.relations[0]
    else:
        return None

# Функция для вычисления длины маршрута в километрах
def calculate_route_distance(relation):
    distance = 0

    for member in relation.members:
        if member.type == "w":
            distance += member.way.length

    return distance / 1000  # Конвертируем в километры

station_data = pd.read_csv('partial_results.csv')
history_data = pd.read_csv('history_data.csv')



# # Получаем отношение маршрута
# relation = get_route_relation(st_id_disl, st_id_dest)

# if relation:
#     # Вычисляем расстояние между станциями через маршрут
#     distance_km = calculate_route_distance(relation)
#     print(f"Расстояние между станциями {st_id_disl} и {st_id_dest} через маршрут: {distance_km:.2f} км")
# else:
#     print("Не удалось найти маршрут между станциями.")


In [6]:
import pandas as pd
history_data = pd.read_csv('history_data.csv')


In [7]:
history_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999999 entries, 0 to 999998
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          999999 non-null  int64  
 1   WAGNUM              999999 non-null  int64  
 2   OPERDATE            999999 non-null  object 
 3   ST_ID_DISL          999999 non-null  int64  
 4   ST_ID_DEST          999999 non-null  int64  
 5   TRAIN_INDEX         999984 non-null  object 
 6   TRAIN_NUM           999984 non-null  float64
 7   WAGNUM_CURRENT      999999 non-null  int64  
 8   OPERATION_DURATION  999984 non-null  object 
 9   ROUTE_STATIONS      999999 non-null  object 
 10  EARLIEST_TIME       999984 non-null  object 
 11  LATEST_TIME         999984 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 91.6+ MB


In [9]:
station_data = pd.read_csv('partial_results.csv')

In [10]:
dist_data = pd.merge(history_data, station_data, how="left", left_on="ST_ID_DISL", right_on="ST_ID")
dist_data = pd.merge(dist_data, station_data, how="left", left_on="ST_ID_DEST", right_on="ST_ID")

In [11]:
dist_data

,Unnamed: 0,WAGNUM,OPERDATE,ST_ID_DISL,ST_ID_DEST,TRAIN_INDEX,TRAIN_NUM,WAGNUM_CURRENT,OPERATION_DURATION,ROUTE_STATIONS,...,ST_ID_x,LATITUDE_x,LONGITUDE_x,NAME_x,OSM_ID_x,ST_ID_y,LATITUDE_y,LONGITUDE_y,NAME_y,OSM_ID_y
0,339038,5100,2023-06-01 00:00:00,1268,1655,1268-677-1663,677.0,2,1 days 03:30:00,1268-1655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,340706,6516,2023-06-01 00:00:00,1268,1655,1268-677-1663,677.0,2,1 days 03:30:00,1268-1655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,339037,5100,2023-06-01 00:00:01,1268,1655,1268-677-1663,677.0,2,1 days 03:30:00,1268-1655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,340705,6516,2023-06-01 00:00:01,1268,1655,1268-677-1663,677.0,2,1 days 03:30:00,1268-1655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,88525,6394,2023-06-01 00:01:00,4576,633,20715-079-20713,79.0,1,0 days 06:47:01,4576-633,...,NaN,NaN,NaN,NaN,NaN,633.0,53.655200,47.106100,Барыш,3.991185e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,596568,9100,2023-09-01 00:00:00,1771,1771,-236-,236.0,7,0 days 08:28:00,1771-1771,...,1771.0,53.615717,66.563803,Аксуат,3.813120e+09,1771.0,53.615717,66.563803,Аксуат,3.813120e+09
999995,56767,5103,2023-09-01 00:00:00,7076,574,17506-012-20716,12.0,3,0 days 09:09:00,7076-574,...,7076.0,45.457400,133.422000,Лесозаводск-2,7.021154e+09,NaN,NaN,NaN,NaN,NaN
999996,101489,4596,2023-09-01 00:00:00,7076,1138,17506-012-20716,12.0,3,0 days 09:09:00,7076-1138,...,7076.0,45.457400,133.422000,Лесозаводск-2,7.021154e+09,NaN,NaN,NaN,NaN,NaN
999997,543961,7349,2023-09-01 00:00:00,1771,1771,-236-,236.0,7,0 days 08:28:00,1771-1771,...,1771.0,53.615717,66.563803,Аксуат,3.813120e+09,1771.0,53.615717,66.563803,Аксуат,3.813120e+09


In [12]:
dist_data.dropna(inplace=True)

In [13]:
dist_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 357822 entries, 5 to 999998
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          357822 non-null  int64  
 1   WAGNUM              357822 non-null  int64  
 2   OPERDATE            357822 non-null  object 
 3   ST_ID_DISL          357822 non-null  int64  
 4   ST_ID_DEST          357822 non-null  int64  
 5   TRAIN_INDEX         357822 non-null  object 
 6   TRAIN_NUM           357822 non-null  float64
 7   WAGNUM_CURRENT      357822 non-null  int64  
 8   OPERATION_DURATION  357822 non-null  object 
 9   ROUTE_STATIONS      357822 non-null  object 
 10  EARLIEST_TIME       357822 non-null  object 
 11  LATEST_TIME         357822 non-null  object 
 12  ST_ID_x             357822 non-null  float64
 13  LATITUDE_x          357822 non-null  float64
 14  LONGITUDE_x         357822 non-null  float64
 15  NAME_x              357822 non-null  ob

In [14]:
# Группируем DataFrame по ST_ID_DISL и ST_ID_DEST
grouped_df = dist_data.groupby(['ST_ID_DISL', 'ST_ID_DEST']).agg({
    'LATITUDE_x': 'first',  # Пример агрегации, может потребоваться другая
    'LONGITUDE_x': 'first',  # Пример агрегации, может потребоваться другая
    'NAME_x': 'first',
    'OSM_ID_x': 'first',
    'LATITUDE_y': 'first',  # Пример агрегации, может потребоваться другая
    'LONGITUDE_y': 'first',  # Пример агрегации, может потребоваться другая
    'NAME_y': 'first',
    'OSM_ID_y': 'first',
}).reset_index()

In [16]:
from geopy.distance import geodesic

grouped_df['DISTANCE'] = None

for index, row in grouped_df.iterrows():
    coordinates_disl = (row['LATITUDE_x'], row['LONGITUDE_x'])
    coordinates_dest = (row['LATITUDE_y'], row['LONGITUDE_y'])

    distance_km = geodesic(coordinates_disl, coordinates_dest).kilometers
    grouped_df.at[index, 'DISTANCE'] = distance_km

# Отображаем DataFrame с расстояниями
grouped_df

,ST_ID_DISL,ST_ID_DEST,LATITUDE_x,LONGITUDE_x,NAME_x,OSM_ID_x,LATITUDE_y,LONGITUDE_y,NAME_y,OSM_ID_y,DISTANCE
0,24,1555,48.6718,43.8269,Мариновка,2.217978e+09,46.801152,61.675400,Арал теңізі,1.802313e+09,1351.566982
1,44,1555,48.2760,46.1972,Ахтуба,7.521689e+08,46.801152,61.675400,Арал теңізі,1.802313e+09,1174.746941
2,50,1555,46.3709,47.9847,Правый Берег,1.829623e+09,46.801152,61.675400,Арал теңізі,1.802313e+09,1049.034827
3,95,95,48.2117,46.8363,Нижний Баскунчак,2.492699e+09,48.211700,46.836300,Нижний Баскунчак,2.492699e+09,0.0
4,95,108,48.2117,46.8363,Нижний Баскунчак,2.492699e+09,49.884200,46.857800,Калашниковский,2.335043e+09,186.006461
...,...,...,...,...,...,...,...,...,...,...,...
2006,22800,1228,53.2137,34.4091,Брянск-Льговский,2.036749e+09,51.160400,43.697800,Три Острова,2.089683e+09,674.502489
2007,22802,1228,48.2413,41.4785,Ковылкин,1.397013e+09,51.160400,43.697800,Три Острова,2.089683e+09,361.957239
2008,23155,1771,60.6247,128.2190,Ханиердах,4.861965e+09,53.615717,66.563803,Аксуат,3.813120e+09,3669.810022
2009,23156,1771,60.0464,127.2040,Карбыкан,4.861956e+09,53.615717,66.563803,Аксуат,3.813120e+09,3633.810015


In [17]:
grouped_df.to_csv('dist_info.csv')

In [13]:
grouped_df

,ST_ID_DISL,ST_ID_DEST,LATITUDE_x,LONGITUDE_x,NAME_x,OSM_ID_x,LATITUDE_y,LONGITUDE_y,NAME_y,OSM_ID_y
0,24,1555,48.6718,43.8269,Мариновка,2.217978e+09,46.801152,61.675400,Арал теңізі,1.802313e+09
1,44,1555,48.2760,46.1972,Ахтуба,7.521689e+08,46.801152,61.675400,Арал теңізі,1.802313e+09
2,50,1555,46.3709,47.9847,Правый Берег,1.829623e+09,46.801152,61.675400,Арал теңізі,1.802313e+09
3,95,95,48.2117,46.8363,Нижний Баскунчак,2.492699e+09,48.211700,46.836300,Нижний Баскунчак,2.492699e+09
4,95,108,48.2117,46.8363,Нижний Баскунчак,2.492699e+09,49.884200,46.857800,Калашниковский,2.335043e+09
...,...,...,...,...,...,...,...,...,...,...
2006,22800,1228,53.2137,34.4091,Брянск-Льговский,2.036749e+09,51.160400,43.697800,Три Острова,2.089683e+09
2007,22802,1228,48.2413,41.4785,Ковылкин,1.397013e+09,51.160400,43.697800,Три Острова,2.089683e+09
2008,23155,1771,60.6247,128.2190,Ханиердах,4.861965e+09,53.615717,66.563803,Аксуат,3.813120e+09
2009,23156,1771,60.0464,127.2040,Карбыкан,4.861956e+09,53.615717,66.563803,Аксуат,3.813120e+09


In [5]:
grouped_df.to_csv('geodata.csv')

NameError: name 'grouped_df' is not defined

In [46]:
import overpass
import pandas as pd

def get_station_info(osm_id):
    api = overpass.API()
    query = f"""
        node({osm_id});
        out meta;
    """

    result = api.Get(query)

    if result['features']:
        tags = result['features'][0].get('properties', {})
        print(tags)
        return tags
    else:
        return None

osm_id_list = station_data[station_data['OSM_ID'].notna()]['OSM_ID'].unique().tolist()

print("Количество OSM_ID:", len(osm_id_list))
data = []

# Заполняем список информацией о станциях
for osm_id in osm_id_list:
    station_info = get_station_info(osm_id)

    if station_info:
        row = {
            'OSM_ID': osm_id,
            'Name': station_info.get('name', ''),
            'Railway': station_info.get('railway', ''),
            'Operator': station_info.get('operator', ''),
            'Network': station_info.get('network', ''),
            'Usage': station_info.get('usage', ''),
            'Public_Transport': station_info.get('public_transport', ''),
            'Addr_Postcode': station_info.get('addr:postcode', ''),
            'Addr_City': station_info.get('addr:city', ''),
            'Addr_Country': station_info.get('addr:country', ''),
            'Website': station_info.get('website', ''),
            'Opening_Hours': station_info.get('opening_hours', '')
        }

        data.append(row)

station_df = pd.DataFrame(data)

print(station_df)


Количество OSM_ID: 7694
{'name': 'Чернышков', 'network': 'РЖД', 'operator': 'Приволжская железная дорога', 'public_transport': 'station', 'railway': 'station', 'train': 'yes', 'uic_name': 'Tschernischkow', 'uic_ref': '2020861'}
{'name': 'Обливская', 'network': 'РЖД', 'operator': 'Приволжская железная дорога', 'public_transport': 'station', 'railway': 'station', 'train': 'yes', 'uic_name': 'Obliwskaja', 'uic_ref': '2020850'}
{'esr:user': '615001', 'name': 'Суровикино', 'network': 'РЖД', 'operator': 'Приволжская жеоезная дорога', 'public_transport': 'station', 'railway': 'station', 'train': 'yes', 'uic_name': 'Surowkino', 'uic_ref': '2020840'}
{'name': 'Чир', 'public_transport': 'station', 'railway': 'station', 'train': 'yes'}
{'name': 'Ложки', 'public_transport': 'station', 'railway': 'station', 'train': 'yes'}
{'esr:user': '615302', 'name': 'Ляпичев', 'name:ru': 'Ляпичев', 'railway': 'station', 'uic_ref': '2020856'}
{'name': 'Кривомузгинская', 'railway': 'station'}
{'name': 'Мариновка'

KeyboardInterrupt: 